In [1]:
import numpy as np
import sympy as sym
import json
from pydae.urisi.urisi_builder import urisi
import pydae.build_cffi as db

In [2]:
data = {"system":{"S_base":1e6, "K_p_agc":0.01, "K_i_agc":0.01},
        "buses":[
                 {"name": "AC1",  "pos_x":   0, "pos_y":   0, "units": "m", "U_kV":20.0, 'N_nodes':3, 'phi_deg_0':30.0},
                 {"name": "AC2",  "pos_x":  30, "pos_y":   0, "units": "m", "U_kV":0.4},
                 {"name": "AC3",  "pos_x":  90, "pos_y":   0, "units": "m", "U_kV":0.4},
                 {"name": "AC4",  "pos_x": 150, "pos_y":   0, "units": "m", "U_kV":0.4},
                 {"name": "DC2",  "pos_x":  30, "pos_y": -20, "units": "m", "U_kV":0.8, "acdc":"DC",'nodes':[0,1]},
                 {"name": "DC4",  "pos_x": 150, "pos_y": -20, "units": "m", "U_kV":0.8, "acdc":"DC",'nodes':[0,1]}
                ],    
        "transformers":[
                        {"bus_j": "AC1",  "bus_k": "AC2",  "S_n_kVA": 1000.0, "U_j_kV":20, "U_k_kV":0.4,
                         "R_cc_pu": 0.01, "X_cc_pu":0.04, "connection": "Dyn11",   "conductors_j": 3, "conductors_k": 4,
                         "monitor":True}
                       ],
        "lines":[
                 {"bus_j": "AC2",  "bus_k": "AC3",  "code":   "UG1", "m": 100.0,"monitor":True},
                 {"bus_j": "AC3",  "bus_k": "AC4",  "code":   "UG1", "m": 100.0,"monitor":True},
                 {"bus_j": "DC2",  "bus_k": "DC4",  "code": "UG1dc", "m": 200.0,"vsc_line":True, "monitor":True,"bus_j_nodes":[0,1],"bus_k_nodes":[0,1]}
                ],
        "loads":[
              #  {"bus": "AC3" , "kVA": 0.0, "pf": 0.85,"type":"3P+N","model":"ZIP"},
                 {"bus": "AC4" , "kVA": 0.0, "pf": 0.85,"type":"3P+N","model":"ZIP"},
                 {"bus": "DC4" , "kW": 1.0,"type":"DC","model":"ZIP"}
                ],
        "shunts":[
                {"bus": "AC2" , "R":  3.0, "X": 0.0, "bus_nodes": [3,0]},
               # {"bus": "AC4" , "R": 40.0, "X": 0.0, "bus_nodes": [3,0]},
                 #{"bus": "DC2" , "R":  0.1, "X": 0.0, "bus_nodes": [1]},
                 #{"bus": "DC4" , "R":  0.1, "X": 0.0, "bus_nodes": [1]}
                 ],
        "genapes":[
               {"type":"ac3ph3w_ideal", "bus": "AC1","S_n":100e3,"U_n":20e3,"R":0.01,"X":0.1,"R_n":0.01,"X_n":0.1,"R_ng":3.0,"X_ng":0.0,"K_p":0.1,"K_agc":1.0,
                             "T_p":0.1,"T_e":0.1,"K_delta":0.001, "T_c":0.1, "Droop":0.05, "T_v":0.1, "K_f":1e-2, "T_f":1e-2},
             #    {"type":"dc_ideal", "bus": "DC2","S_n":100e3,"U_n":800.0,"R_s":0.01,"R_g":3.0},
                   ],
"vscs":[
       # {"bus": "AC1", "type":"ac_3ph_4w_l","S_n":100e3,"U_n":400.0,"R":0.01,"X":0.1,"R_n":0.01,"X_n":0.1,"R_ng":3.0,"X_ng":0.0},
        {"bus_ac":  "AC2", "bus_dc":"DC2", "type":"acdc_3ph_4w_vdc_q","a":2.92,"b":0.45,"c":0.027},
        {"bus_ac":  "AC4", "bus_dc":"DC4", "type":"acdc_3ph_4w_pq","a":2.92,"b":0.45,"c":0.027},
      #  {"bus_ac":  "AC4", "bus_dc":"DC4", "type":  "ac3ph4wpq","a":2.92,"b":0.45,"c":0.027}
       ],
        "line_codes":
            {
        "UG1":
            {"R":[[ 0.211,  0.049,  0.049,  0.049],
                  [ 0.049,  0.211,  0.049,  0.049],
                  [ 0.049,  0.049,  0.211,  0.049],
                  [ 0.049,  0.049,  0.049,  0.211]],
             "X":[[ 0.747,  0.673,  0.651,  0.673],
                  [ 0.673,  0.747,  0.673,  0.651],
                  [ 0.651,  0.673,  0.747,  0.673],
                  [ 0.673,  0.651,  0.673,  0.747]], "I_max":430.0
            },
       "UG1dc":
        {"R":[[ 0.211,  0.0],
              [ 0.0,  0.211]],
         "X":[[ 0.0,  0.0],
              [ 0.0,  0.0]], "I_max":430.0
                }
 }}


In [3]:
u = urisi(data)
u.construct('acdc')
u.verbose = True
u.compile('acdc')

[{'bus': 'AC4', 'kVA': 0.0, 'pf': 0.85, 'type': '3P+N', 'model': 'ZIP'}, {'bus': 'DC4', 'kW': 1.0, 'type': 'DC', 'model': 'ZIP'}]
check_system (time: 0.0)
computing jacobians Fx_run,Fy_run  (time: 0.010 s)
computing jacobians Gx_run,Gy_run  (time: 0.015 s)
computing jacobians Fu_run,Gu_run  (time: 0.732 s)
computing jacobians Fx_ini,Fy_ini  (time: 0.853 s)
computing jacobians Gx_ini,Gy_ini  (time: 0.855 s)
computing jacobians Hx_run,Hy_run,Hu_run  (time: 1.0328152179718018 s)
end system  (time: 1.424 s)
computing jac_ini (time: 0.0)
computing jac_run (time: 0.0020003318786621094)
computing jac_trap (time: 0.004000186920166016)
end of jacobians computation (time: 0.008)
writting f_ini and g_ini code (time: 0.000 s)
writting f_run and g_run code (time: 0.253 s)
writting h_run code (time: 0.533 s)
converting jac_ini to sp_jac_ini  (time: 0.674 s)
running sym2rhs for sp_jac_ini (time: 0.676 s)
converting jac_run to sp_jac_run  (time: 0.974 s)
running sym2rhs for sp_jac_run (time: 0.977 s)


In [4]:
import acdc

In [5]:
model = acdc.model()
model.ini({"K_p_agc":0.0, "K_i_agc":0.0,"K_delta_AC1":1.0,"K_agc_AC1":0.0,"K_xif":0.01,
             "p_load_AC4_a":-10e3/3,"p_load_AC4_b":-10e3/3,"p_load_AC4_c":-10e3/3,
             "p_DC4":100e3 },'xy_0.json')
model.report_x()
model.report_y()
model.report_z()


Input or parameter K_delta_AC1 not found.
Input or parameter K_agc_AC1 not found.
xi_freq =  0.00
V_AC1_0_r = 11546.98
V_AC1_0_i = -0.27
V_AC1_1_r = -5773.72
V_AC1_1_i = -9999.84
V_AC1_2_r = -5773.26
V_AC1_2_i = 10000.11
V_AC2_0_r = 199.38
V_AC2_0_i = -116.11
V_AC2_1_r = -200.24
V_AC2_1_i = -114.61
V_AC2_2_r =  0.87
V_AC2_2_i = 230.72
V_AC2_3_r =  0.00
V_AC2_3_i = -0.00
V_AC3_0_r = 199.66
V_AC3_0_i = -116.14
V_AC3_1_r = -200.39
V_AC3_1_i = -114.82
V_AC3_2_r =  0.75
V_AC3_2_i = 230.93
V_AC3_3_r = -0.02
V_AC3_3_i =  0.03
V_AC4_0_r = 199.95
V_AC4_0_i = -116.16
V_AC4_1_r = -200.54
V_AC4_1_i = -115.03
V_AC4_2_r =  0.62
V_AC4_2_i = 231.13
V_AC4_3_r = -0.03
V_AC4_3_i =  0.05
V_DC2_0_r = 400.00
V_DC2_0_i = -0.00
V_DC2_1_r = -400.00
V_DC2_1_i = -0.00
V_DC4_0_r = 394.65
V_DC4_0_i = -0.00
V_DC4_1_r = -394.65
V_DC4_1_i = -0.00
i_load_AC4_a_r = 12.46
i_load_AC4_a_i = -7.24
i_load_AC4_b_r = -12.50
i_load_AC4_b_i = -7.18
i_load_AC4_c_r =  0.04
i_load_AC4_c_i = 14.42
i_load_AC4_n_r =  0.00
i_load_AC4_

In [6]:
for it,y in enumerate(model.y_ini_list):
    print(f'{it}, {y}: {np.sum(model.jac_ini[it,:])}')

0, V_AC1_0_r: -1.01
1, V_AC1_0_i: -1.0
2, V_AC1_1_r: -1.0
3, V_AC1_1_i: -0.9999999999999999
4, V_AC1_2_r: -1.0
5, V_AC1_2_i: -0.9999999999999998
6, V_AC2_0_r: -1.0
7, V_AC2_0_i: -1.0
8, V_AC2_1_r: -1.0000000000000142
9, V_AC2_1_i: -0.9999999999999716
10, V_AC2_2_r: -1.0000000000000568
11, V_AC2_2_i: -1.000000000000007
12, V_AC2_3_r: -1.0
13, V_AC2_3_i: -1.0
14, V_AC3_0_r: -1.0000000000000853
15, V_AC3_0_i: 0.0
16, V_AC3_1_r: 0.0
17, V_AC3_1_i: 0.0
18, V_AC3_2_r: 0.0
19, V_AC3_2_i: 0.0
20, V_AC3_3_r: 0.0
21, V_AC3_3_i: 0.0
22, V_AC4_0_r: 0.0
23, V_AC4_0_i: -1.0
24, V_AC4_1_r: -1.0
25, V_AC4_1_i: -1.0
26, V_AC4_2_r: -1.0
27, V_AC4_2_i: -1.0
28, V_AC4_3_r: -1.0
29, V_AC4_3_i: -1.0
30, V_DC2_0_r: -1.0
31, V_DC2_0_i: -1.0
32, V_DC2_1_r: 0.0010000000000012221
33, V_DC2_1_i: -1.0
34, V_DC4_0_r: 0.0010000000000012221
35, V_DC4_0_i: 0.0
36, V_DC4_1_r: 0.0010000000000012221
37, V_DC4_1_i: -2.0
38, i_load_DC4_p_r: 0.0010000000000012221
39, p_a_d_AC2: 798.9448346562143
40, p_b_d_AC2: -0.6666666666

In [28]:
model.report_z()

i_l_AC2_0_AC3_0_r =  0.00
i_l_AC2_0_AC3_0_i =  0.00
i_l_AC2_1_AC3_1_r =  0.00
i_l_AC2_1_AC3_1_i =  0.00
i_l_AC2_2_AC3_2_r =  0.00
i_l_AC2_2_AC3_2_i =  0.00
i_l_AC2_3_AC3_3_r =  0.00
i_l_AC2_3_AC3_3_i =  0.00
i_l_AC3_0_AC4_0_r =  0.00
i_l_AC3_0_AC4_0_i =  0.00
i_l_AC3_1_AC4_1_r =  0.00
i_l_AC3_1_AC4_1_i =  0.00
i_l_AC3_2_AC4_2_r =  0.00
i_l_AC3_2_AC4_2_i =  0.00
i_l_AC3_3_AC4_3_r =  0.00
i_l_AC3_3_AC4_3_i =  0.00
i_l_DC2_0_DC4_0_r =  0.00
i_l_DC2_0_DC4_0_i =  0.00
i_l_DC2_1_DC4_1_r =  0.00
i_l_DC2_1_DC4_1_i =  0.00
i_t_AC1_0_r =  0.00
i_t_AC1_0_i =  0.00
i_t_AC1_1_r =  0.00
i_t_AC1_1_i =  0.00
i_t_AC1_2_r =  0.00
i_t_AC1_2_i =  0.00
i_t_AC2_0_r =  0.00
i_t_AC2_0_i =  0.00
i_t_AC2_1_r =  0.00
i_t_AC2_1_i =  0.00
i_t_AC2_2_r =  0.00
i_t_AC2_2_i =  0.00
i_vsc_AC2_a_m =  0.00
i_vsc_AC2_b_m =  0.00
i_vsc_AC2_c_m =  0.00
i_vsc_AC2_n_m =  0.00
p_vsc_AC2 =  0.00
p_vsc_loss_AC2 =  0.00
i_vsc_AC4_a_m =  0.00
i_vsc_AC4_b_m =  0.00
i_vsc_AC4_c_m =  0.00
i_vsc_AC4_n_m =  0.00
p_vsc_AC4 =  0.00
p_vsc

In [9]:
model.y_ini_list

['V_DC2_0_r',
 'V_DC2_0_i',
 'V_DC2_1_r',
 'V_DC2_1_i',
 'V_DC4_0_r',
 'V_DC4_0_i',
 'V_DC4_1_r',
 'V_DC4_1_i',
 'i_load_DC4_p_r',
 'i_vsc_DC2_sp',
 'i_vsc_DC2_sn',
 'v_og_DC2',
 'omega_coi',
 'p_agc']

In [10]:
model.ini({"p_load_B2_a":10e3,"p_load_B2_b":10e3,"p_load_B2_c":10E3},'xy_0.json')
model.report_y()

Input or parameter p_load_B2_a not found.
Input or parameter p_load_B2_b not found.
Input or parameter p_load_B2_c not found.
V_DC2_0_r = 398.73
V_DC2_0_i = -0.00
V_DC2_1_r = -398.73
V_DC2_1_i = -0.00
V_DC4_0_r = 393.36
V_DC4_0_i = -0.00
V_DC4_1_r = -393.36
V_DC4_1_i = -0.00
i_load_DC4_p_r = 127.11
i_vsc_DC2_sp = 127.11
i_vsc_DC2_sn = -127.11
v_og_DC2 =  0.00
omega_coi =  1.00
p_agc =  0.00


In [11]:
%%timeit 

for load in np.arange(0,200e3,10e3):
    model.spini({"p_load_B1_a":10e3,"p_load_B1_b":10e3,"p_load_B1_c":load},'xy_0.json')


Input or parameter p_load_B1_a not found.
Input or parameter p_load_B1_b not found.
Input or parameter p_load_B1_c not found.


AttributeError: 'model' object has no attribute 'P_d'

In [10]:
%%timeit 

for load in np.arange(0,200e3,10e3):
    model.ini({"p_load_B1_a":10e3,"p_load_B1_b":10e3,"p_load_B1_c":load},'xy_0.json')

3.71 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
